In [1]:
import pandas as pd
import pickle
from numpy import savetxt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Params
target = 'Survived'
features = ['Sex', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
idx = 'Passengerid'

# Paths 
data_train_path = "/home/walter/Documents/personal_projects/new-titan/data/raw/train.csv"

train_dict_path = "/home/walter/Documents/personal_projects/new-titan/notebooks/experiments/exp_02/checkpoint/train_dict.pkl"
pclass_encoder_path = "/home/walter/Documents/personal_projects/new-titan/notebooks/experiments/exp_02/checkpoint/pclass_encoder.pkl"
embarked_encoder_path = "/home/walter/Documents/personal_projects/new-titan/notebooks/experiments/exp_02/checkpoint/embarked_encoder.pkl"
sex_encoder_path = "/home/walter/Documents/personal_projects/new-titan/notebooks/experiments/exp_02/checkpoint/sex_encoder.pkl"

tunned_model_path = '/home/walter/Documents/personal_projects/new-titan/notebooks/experiments/exp_02/artifacts/tunned_model.pkl'
trained_model_path = '/home/walter/Documents/personal_projects/new-titan/notebooks/experiments/exp_02/artifacts/model.pkl'

In [2]:
def divide_for_train(df, test_size):
    X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=test_size, random_state=100)
    label_train = df.iloc[X_train.index, 0]
    label_test = df.iloc[X_test.index, 0]

    return X_train, y_train, label_train, X_test, y_test, label_test

def train_pclass_encoder(df, path):
    enc = OneHotEncoder(sparse=False, handle_unknown='infrequent_if_exist')
    enc.fit(df[['Pclass']])

    file = open(path, 'wb')
    pickle.dump(enc, file)
    file.close()

def train_embarked_encoder(df, path):
    enc = OneHotEncoder(sparse=False, handle_unknown='infrequent_if_exist')
    enc.fit(df[['Embarked']])

    file = open(path, 'wb')
    pickle.dump(enc, file)
    file.close()

def train_sex_encoder(df, path):
    enc = LabelEncoder()
    enc.fit(df['Sex'])

    file = open(path, 'wb')
    pickle.dump(enc, file)
    file.close()

def apply_sex_encoder(df, path):
    file = open(path, 'rb')
    enc = pickle.load(file)
    file.close()

    out = enc.transform(df['Sex'])
    df['Sex'] = out
    return df

def apply_pclass_encoder(df, path):
    file = open(path, 'rb')
    enc = pickle.load(file)
    file.close()

    new_cols = ['Pclass_' + str(c) for c in enc.categories_[0]]
    df[new_cols] = enc.transform(df[['Pclass']])
    df = df.drop(['Pclass'], axis=1)
    return df

def apply_embarked_encoder(df, path):
    file = open(path, 'rb')
    enc = pickle.load(file)
    file.close()

    new_cols = ['Embarked_' + str(c) for c in enc.categories_[0]]
    df[new_cols] = enc.transform(df[['Embarked']])
    df = df.drop(['Embarked'], axis=1)
    return df

def save_train_dict(obj, path):
    file = open(path, 'wb')
    pickle.dump(obj, file)
    file.close()

def load_train_dict(path):
    file = open(path, 'rb')
    dict = pickle.load(file)
    file.close()
    return dict

def load_model(path):
    pkl_file = open(path, 'rb')
    selected_model = pickle.load(pkl_file)
    pkl_file.close()

    return selected_model

def save_train_files():
    savetxt('/home/walter/Documents/personal_projects/new-titan/data/processed/data_train/X_train.csv', X_train, delimiter=',')
    savetxt('/home/walter/Documents/personal_projects/new-titan/data/processed/data_train/y_train.csv', y_train, delimiter=',')
    savetxt('/home/walter/Documents/personal_projects/new-titan/data/processed/data_train/label_train.csv', label_train, delimiter=',')
    savetxt('/home/walter/Documents/personal_projects/new-titan/data/processed/data_test/label_test.csv', label_test, delimiter=',')
    savetxt('/home/walter/Documents/personal_projects/new-titan/data/processed/data_test/X_test.csv', X_test, delimiter=',')
    savetxt('/home/walter/Documents/personal_projects/new-titan/data/processed/data_test/y_test.csv', y_test, delimiter=',')

In [3]:
# load
data = pd.read_csv(data_train_path, header=0)

# Preprocessing
X_train = data[features]
y_train = data[target]

# apply imps
train_dict = load_train_dict(train_dict_path)
X_train['Age'] = X_train['Age'].fillna(train_dict['age_mean'])
X_train['Embarked'] = X_train['Embarked'].fillna(train_dict['embarked_mode'])
X_train['Fare'] = X_train['Fare'].fillna(train_dict['fare_mean'])

# apply encoders
X_train = apply_pclass_encoder(X_train,pclass_encoder_path)
X_train = apply_sex_encoder(X_train, sex_encoder_path)
X_train = apply_embarked_encoder(X_train, embarked_encoder_path)

/tmp/ipykernel_57462/960873771.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Age'] = X_train['Age'].fillna(train_dict['age_mean'])
/tmp/ipykernel_57462/960873771.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Embarked'] = X_train['Embarked'].fillna(train_dict['embarked_mode'])
/tmp/ipykernel_57462/960873771.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [4]:
# fit
model = load_model(tunned_model_path)
model.fit(X_train, y_train)

# save model
with open(trained_model_path, 'wb') as out:
    pickle.dump(model, out)